In [1]:
# !pip install wheel
# !pip install h5py
# !pip install torch==1.10.0+cu113 torchvision==0.11.1+cu113 torchaudio==0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

In [2]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import h5py
import yaml
import pickle
import os
import argparse
import random
from tqdm.notebook import tqdm
from collections import Counter
from dataclasses import dataclass, field
import contextlib
import math

In [3]:
data = pd.read_csv('../processed_data/utrs.csv')
data = data[['Gene Name', 'foreign']].dropna(axis=0).rename(columns={'Gene Name': 'gene', 'foreign':'seq'})

In [4]:
data.head()

,gene,seq
4,VPS8,ACATTTCTAAATATTTAATACAACTTTGGTTACATAAAAGTAAAAT...
5,SSA1,AGCCAATTGGTGCGGCAATTGATAATAACGAAAATGTCTTTTAATG...
6,ERP2,AGAACTTTTCAATCTACGAAAAATATATGTCCGCAATATAGAACAC...
7,FUN14,AGCAAGACAAATGACCAGATATAAACGAGGGTTATATTCTTTCGTT...
8,SPO7,AAAGAGTTGGAGGGCTTCTTCCTTCGAATAAGAGGTCATATTTACC...


In [91]:
from itertools import product

In [92]:

def entropy(string):
    "Calculates the Shannon entropy of a string"

    # get probability of chars in string
    prob = [ float(string.count(c)) / len(string) for c in dict.fromkeys(list(string)) ]

    # calculate the entropy
    entropy = - sum([ p * math.log(p) / math.log(2.0) for p in prob ])

    return min(1, abs(entropy) / 2)

In [93]:
# Efficiency element
eff_el1 = "TATATA"
eff_el2 = "TTTTTATA"
eff_ctrl = "GCGCGC"
# Mutational scan of efficiency element?
# Positioning element
pos_el = "AAWAAA"
# Puf protein binding sites
puf1_2 = "TAATNNNTAAT"
puf3 = "TGTANATA"
puf4 = "TGTANANTA"
puf5 = "TGTANNNNTA"
puf6 = "TTGT"
# Poly-T sequences
poly_t = "TTTTTTTT"
_kmers = [eff_el1, eff_el2, eff_ctrl,
           pos_el,
           puf1_2, puf3, puf4, puf5, puf6,
           poly_t]
specific_kmers = []

for kmer in _kmers:
    specific_kmers.extend([''.join(y) for y in list(product(*(['A', 'T', 'G', 'C'] if x=='N'  else (
        ['A', 'T'] if x=='W' else  (x,)) for x in kmer)))])

In [94]:
def compute_stride(seq):
    return max(1, round(len(seq) * (1 - entropy(seq))))

In [96]:
search_strides = [compute_stride(s) for s in specific_kmers]
list(map(tuple, zip(*[specific_kmers,search_strides])))

[('TATATA', 3),
 ('TTTTTATA', 5),
 ('GCGCGC', 3),
 ('AAAAAA', 6),
 ('AATAAA', 4),
 ('TAATAAATAAT', 6),
 ('TAATAATTAAT', 6),
 ('TAATAAGTAAT', 4),
 ('TAATAACTAAT', 4),
 ('TAATATATAAT', 6),
 ('TAATATTTAAT', 6),
 ('TAATATGTAAT', 4),
 ('TAATATCTAAT', 4),
 ('TAATAGATAAT', 4),
 ('TAATAGTTAAT', 4),
 ('TAATAGGTAAT', 3),
 ('TAATAGCTAAT', 2),
 ('TAATACATAAT', 4),
 ('TAATACTTAAT', 4),
 ('TAATACGTAAT', 2),
 ('TAATACCTAAT', 3),
 ('TAATTAATAAT', 6),
 ('TAATTATTAAT', 6),
 ('TAATTAGTAAT', 4),
 ('TAATTACTAAT', 4),
 ('TAATTTATAAT', 6),
 ('TAATTTTTAAT', 6),
 ('TAATTTGTAAT', 4),
 ('TAATTTCTAAT', 4),
 ('TAATTGATAAT', 4),
 ('TAATTGTTAAT', 4),
 ('TAATTGGTAAT', 3),
 ('TAATTGCTAAT', 2),
 ('TAATTCATAAT', 4),
 ('TAATTCTTAAT', 4),
 ('TAATTCGTAAT', 2),
 ('TAATTCCTAAT', 3),
 ('TAATGAATAAT', 4),
 ('TAATGATTAAT', 4),
 ('TAATGAGTAAT', 3),
 ('TAATGACTAAT', 2),
 ('TAATGTATAAT', 4),
 ('TAATGTTTAAT', 4),
 ('TAATGTGTAAT', 3),
 ('TAATGTCTAAT', 2),
 ('TAATGGATAAT', 3),
 ('TAATGGTTAAT', 3),
 ('TAATGGGTAAT', 2),
 ('TAATGGCTAAT'

In [97]:
from heapq import merge
def search_kmer_adaptive(seq, kmer, stride):
    found = []
    pos = 0
    while pos < len(seq):
        if seq[pos:pos+len(kmer)] == kmer:
            found.append(pos)
            pos += stride
        else:
            pos += 1
    return found
from math import floor
def get_kmers_adaptive(seq, k, min_stride):
    kmers = []
    pos = 0
    while pos < len(seq):
        if pos + k > len(seq):
            break
        kmers.append(seq[pos:pos+k])
        pos += max(min_stride, compute_stride(kmers[-1]))
    return kmers

def split(seq, stride):
    ret = [seq[c:c+seqk] for c in np.arange(0, len(seq) ,stride)]
    ret = [y for y in ret if len(y)==seqk]

def search_all_kmers(seq, specific_kmers, kmer_size, min_stride):
    specific_kmers = sorted(specific_kmers, key=len)
    pairs = []
    for kmer in specific_kmers:
        pos = search_kmer_adaptive(seq, kmer, compute_stride(kmer))
        new_words_len = [len(kmer) for _ in range(len(pos))]
        new_words = [kmer for _ in range(len(pos))]
        new_pairs = list(map(tuple, zip(*[pos,new_words_len,new_words]))) 
        pairs = merge(pairs, new_pairs)
    pairs = [(x[0],x[0] + x[1], x[2]) for x in list(pairs)]
    final_sequence = []
    if not pairs:
        return get_kmers_adaptive(seq, kmer_size, min_stride)
    final_sequence =  get_kmers_adaptive(seq[:pairs[0][0]], kmer_size, min_stride)
    for cnt in range(len(pairs) - 1):
        final_sequence.append(pairs[cnt][2])
        final_sequence.extend(get_kmers_adaptive(seq[pairs[cnt][1]: pairs[cnt+1][0]], kmer_size, min_stride))
    final_sequence.append(pairs[-1][2])
    final_sequence.extend(get_kmers_adaptive(seq[pairs[-1][1]:], kmer_size, min_stride))
    return final_sequence

In [98]:
data.seq.iloc[0]

'ACATTTCTAAATATTTAATACAACTTTGGTTACATAAAAGTAAAATTTATACACCTCATTTCATTATGTAGATTCATATATAGAATACCAATTATGATTG'

In [99]:
search_all_kmers(data.seq.iloc[6], specific_kmers, 8, 4)

['AGGTGTTC',
 'GTTCGGTT',
 'GGTTACTT',
 'ACTTTATT',
 'TATTCTGC',
 'CTGCTTTA',
 'TTTAACGC',
 'ACGCCATT',
 'CATTATGA',
 'ATGATTAT',
 'TTATACAC',
 'TTGT',
 'ATTACTTA',
 'CTTATTTT',
 'TTTTTTAA',
 'TGTATATTA',
 'AAACCTTT',
 'CTTTATTT',
 'ATTTTATT',
 'ATTTCACA',
 'CACATTAC',
 'TTACTCAT',
 'TCATCATG',
 'CATGTGGA',
 'TGGAGTAC',
 'GTACTGGA',
 'TTGT']

In [102]:
from tqdm.notebook import tqdm
tqdm.pandas()
KMER_SIZE = 6
preprocessed_seq = data.seq.progress_apply(search_all_kmers, specific_kmers=specific_kmers, kmer_size=KMER_SIZE, min_stride=KMER_SIZE//2)
corpus = [y  for x in preprocessed_seq for y in x]

  0%|          | 0/4870 [00:00<?, ?it/s]

In [103]:
%%writefile gloveConfig.yaml
# first step parameters
## size of the context window
window_size: 20
## maximum vocab_size
vocab_size: 5000
## the number of paritions to divide cooccurence matrix in 
num_partitions: 10
## chunk size of h5py.Dataset
chunk_size: 1000000

# when used in first step, specify the output directory of cooccurrence entries
# when used in second step, specify where to read cooccurrence entries from
cooccurrence_dir: ../processed_data/utrs/cooccurence

# second step parameters
## output path for the trained word vectors 
output_filepath: ../model/utrs/gloveModel
## pytorch training parameters
batch_size: 128
num_epochs: 100
device: cuda
learning_rate: 0.1
## glove paremeters
embedding_size: 30
x_max: 100
alpha: 0.75

Overwriting gloveConfig.yaml


In [104]:
def load_config():
    config_filepath = "gloveConfig.yaml"
    with open(config_filepath, 'r') as f:
        config_dict = yaml.load(f, Loader=yaml.FullLoader)
    config = argparse.Namespace()
    for key, value in config_dict.items():
        setattr(config, key, value)
    return config

In [105]:

@dataclass
class Vocabulary:
    token2index: dict = field(default_factory=dict)
    index2token: dict = field(default_factory=dict)
    token_counts: list = field(default_factory=list)
    _unk_token: int = field(init=False, default=-1)
    
    def add(self, token):
        if token not in self.token2index:
            index = len(self)
            self.token2index[token] = index
            self.index2token[index] = token
            self.token_counts.append(0)
        self.token_counts[self.token2index[token]] += 1
    
    def get_topk_subset(self, k):
        tokens = sorted(
            list(self.token2index.keys()),
            key=lambda token: self.token_counts[self[token]],
            reverse=True
        )
        return type(self)(
            token2index={token: index for index, token in enumerate(tokens[:k])},
            index2token={index: token for index, token in enumerate(tokens[:k])},
            token_counts=[
                self.token_counts[self.token2index[token]] for token in tokens[:k]
            ]
        )
    
    def shuffle(self):
        new_index = [_ for _ in range(len(self))]
        random.shuffle(new_index)
        new_token_counts = [None] * len(self)
        for token, index in zip(list(self.token2index.keys()), new_index):
            new_token_counts[index] = self.token_counts[self[token]]
            self.token2index[token] = index
            self.index2token[index] = token
        self.token_counts = new_token_counts

    def get_index(self, token):
        return self[token]
    
    def get_token(self, index):
        if not index in self.index2token:
            raise Exception("Invalid index.")
        return self.index2token[index]
    
    @property
    def unk_token(self):
        return self._unk_token
    
    def __getitem__(self, token):
        if token not in self.token2index:
            return self._unk_token
        return self.token2index[token]
    
    def __len__(self):
        return len(self.token2index)

In [106]:
@dataclass
class Vectorizer:
    vocab: Vocabulary

    @classmethod
    def from_corpus(cls, corpus, vocab_size):
        vocab = Vocabulary()
        for token in corpus:
            vocab.add(token)
        vocab_subset = vocab.get_topk_subset(vocab_size)
        vocab_subset.shuffle()
        return cls(vocab_subset)

    def vectorize(self, corpus):
        return [self.vocab[token] for token in corpus]

In [107]:
@dataclass
class CooccurrenceEntries:
    vectorized_corpus: list
    vectorizer: Vectorizer
    
    @classmethod
    def setup(cls, corpus, vectorizer):
        return cls(
            vectorized_corpus=vectorizer.vectorize(corpus),
            vectorizer=vectorizer
        )
    
    def validate_index(self, index, lower, upper):
        is_unk = index == self.vectorizer.vocab.unk_token
        if lower < 0:
            return not is_unk
        return not is_unk and index >= lower and index <= upper

    def build(
        self,
        window_size,
        num_partitions,
        chunk_size,
        output_directory="."
    ):
        partition_step = max(1, len(self.vectorizer.vocab) // num_partitions)
        split_points = [0]
        while split_points[-1] + partition_step <= len(self.vectorizer.vocab):
            split_points.append(split_points[-1] + partition_step)
        split_points[-1] = len(self.vectorizer.vocab)

        for partition_id in tqdm(range(len(split_points) - 1)):
            index_lower = split_points[partition_id]
            index_upper = split_points[partition_id + 1] - 1
            cooccurr_counts = Counter()
            for i in tqdm(range(len(self.vectorized_corpus))):
                if not self.validate_index(
                    self.vectorized_corpus[i],
                    index_lower,
                    index_upper
                ):
                    continue
                
                context_lower = max(i - window_size, 0)
                context_upper = min(i + window_size + 1, len(self.vectorized_corpus))
                for j in range(context_lower, context_upper):
                    if i == j or not self.validate_index(
                        self.vectorized_corpus[j],
                        -1,
                        -1
                    ):
                        continue
                    cooccurr_counts[(self.vectorized_corpus[i], self.vectorized_corpus[j])] += 1 / abs(i - j)

            cooccurr_dataset = np.zeros((len(cooccurr_counts), 3))
            for index, ((i, j), cooccurr_count) in enumerate(cooccurr_counts.items()):
                cooccurr_dataset[index] = (i, j, cooccurr_count)
            if partition_id == 0:
                os.makedirs(output_directory, exist_ok=True)
                path = os.path.join(
                        output_directory,
                        "cooccurrence.hdf5"
                    )
                if os.path.isfile(path):
                    os.remove(path)
                file = h5py.File(
                    path,
                    "w"
                )
                dataset = file.create_dataset(
                    "cooccurrence",
                    (len(cooccurr_counts), 3),
                    maxshape=(None, 3),
                    chunks=(chunk_size, 3)
                )
                prev_len = 0
            else:
                prev_len = dataset.len()
                dataset.resize(dataset.len() + len(cooccurr_counts), axis=0)
            dataset[prev_len: dataset.len()] = cooccurr_dataset
        
        file.close()
        with open(os.path.join(output_directory, "vocab.pkl"), "wb") as file:
            pickle.dump(self.vectorizer.vocab, file)

In [108]:
config = load_config()
vectorizer = Vectorizer.from_corpus(
    corpus=corpus,
    vocab_size=min(config.vocab_size, len(set(corpus)))
)
cooccurrence = CooccurrenceEntries.setup(
    corpus=corpus,
    vectorizer=vectorizer
)
cooccurrence.build(
    window_size=config.window_size,
    num_partitions=config.num_partitions,
    chunk_size=config.chunk_size,
    output_directory=config.cooccurrence_dir
) 

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/370167 [00:00<?, ?it/s]

  0%|          | 0/370167 [00:00<?, ?it/s]

  0%|          | 0/370167 [00:00<?, ?it/s]

  0%|          | 0/370167 [00:00<?, ?it/s]

...

  0%|          | 0/370167 [00:00<?, ?it/s]

  0%|          | 0/370167 [00:00<?, ?it/s]

  0%|          | 0/370167 [00:00<?, ?it/s]

  0%|          | 0/370167 [00:00<?, ?it/s]

  0%|          | 0/370167 [00:00<?, ?it/s]

In [109]:
@dataclass
class CooccurrenceDataset(torch.utils.data.Dataset):
    token_ids: torch.Tensor
    cooccurr_counts: torch.Tensor

    def __getitem__(self, index):
        return [self.token_ids[index], self.cooccurr_counts[index]]
    
    def __len__(self):
        return self.token_ids.size()[0]

    
@dataclass
class HDF5DataLoader:
    filepath: str
    dataset_name: str
    batch_size: int
    device: str
    dataset: h5py.Dataset = field(init=False)

    def iter_batches(self):
        chunks = list(self.dataset.iter_chunks())
        random.shuffle(chunks)
        for chunk in chunks:
            chunked_dataset = self.dataset[chunk]
            dataloader = torch.utils.data.DataLoader(
                dataset=CooccurrenceDataset(
                    token_ids=torch.from_numpy(chunked_dataset[:,:2]).long(),
                    cooccurr_counts=torch.from_numpy(chunked_dataset[:,
                        2]).float()
                ),
                batch_size=self.batch_size,
                shuffle=True,
                pin_memory=True
            )
            for batch in dataloader:
                batch = [_.to(self.device) for _ in batch]
                yield batch

    @contextlib.contextmanager
    def open(self):
        with h5py.File(self.filepath, "r") as file:
            self.dataset = file[self.dataset_name]
            yield

In [110]:
class GloVe(nn.Module):

    def __init__(self, vocab_size, embedding_size, x_max, alpha):
        super().__init__()
        self.weight = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=embedding_size,
            sparse=True
        )
        self.weight_tilde = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=embedding_size,
            sparse=True
        )
        self.bias = nn.Parameter(
            torch.randn(
                vocab_size,
                dtype=torch.float,
            )
        )
        self.bias_tilde = nn.Parameter(
            torch.randn(
                vocab_size,
                dtype=torch.float,
            )
        )
        self.weighting_func = lambda x: (x / x_max).float_power(alpha).clamp(0, 1)
    
    def forward(self, i, j, x):
        loss = torch.mul(self.weight(i), self.weight_tilde(j)).sum(dim=1)
        loss = (loss + self.bias[i] + self.bias_tilde[j] - x.log()).square()
        loss = torch.mul(self.weighting_func(x), loss).mean()
        return loss

In [ ]:
config = load_config()
dataloader = HDF5DataLoader(
    filepath=os.path.join(config.cooccurrence_dir, "cooccurrence.hdf5"),
    dataset_name="cooccurrence",
    batch_size=config.batch_size,
    device=config.device
)
model = GloVe(
    vocab_size=min(config.vocab_size, len(set(corpus))),
    embedding_size=config.embedding_size,
    x_max=config.x_max,
    alpha=config.alpha
)
model.to(config.device)
optimizer = torch.optim.Adagrad(
    model.parameters(),
    lr=config.learning_rate
)
losses = []


with dataloader.open():
    model.train()
    
    for epoch in tqdm(range(config.num_epochs)):
        epoch_loss = 0
        for batch in tqdm(dataloader.iter_batches()):
            loss = model(
                batch[0][:, 0],
                batch[0][:, 1],
                batch[1]
            )
            epoch_loss += loss.detach().item()
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        losses.append(epoch_loss)
        print(f"Epoch {epoch}: loss = {epoch_loss}")
        os.makedirs(os.sep.join(config.output_filepath.split(os.sep)[:-1]),exist_ok=True)
        torch.save(model.state_dict(), config.output_filepath)

  0%|          | 0/100 [00:00<?, ?it/s]

0it [00:00, ?it/s]

Epoch 0: loss = 3307.627468649866


0it [00:00, ?it/s]

Epoch 1: loss = 1073.3032392118398


...

0it [00:00, ?it/s]

Epoch 97: loss = 436.50687156936294


0it [00:00, ?it/s]

Epoch 98: loss = 436.38023794516903


0it [00:00, ?it/s]

In [ ]:
plt.plot(losses)

In [ ]:
vectors = (model.weight.weight.to('cpu').detach() + model.weight_tilde.weight.to('cpu').detach()).numpy()

In [ ]:
vectors.shape

In [ ]:
data['kmers_vec'] = [vectorizer.vectorize(x) for x in preprocessed_seq]

In [ ]:
data['kmers_vec']

In [ ]:
data['kmers_enc'] = [np.sum(vectors[[y for y in x if y!=-1], :], axis=0) for x in data['kmers_vec']]

In [ ]:
gloveUtrs = np.vstack(data['kmers_enc'])

In [ ]:
gloveUtrs.shape

In [ ]:
np.isfinite(gloveUtrs).all()

In [ ]:
fDf = pd.read_csv('../processed_data/fluorescence_dataset.csv')
fDf.dropna(subset=['intensity'],inplace=True)
fDf.head()

In [ ]:
whole = data[['gene', 'kmers_enc']].merge(fDf[['gene','intensity']], on='gene', how='inner')
whole.head()

In [ ]:
i = whole['intensity'].copy()
mask = i > 0 # negative intensities make no sense
y = np.log(i[mask])
x = np.vstack(whole['kmers_enc'])[mask]

x = x[y>-20,:]
y = y[y>-20]

In [ ]:
import seaborn as sns
sns.histplot(y)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
pcs = PCA(2).fit_transform(StandardScaler().fit_transform(x))

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.scatter(pcs[:, 0], pcs[:, 1], c=y, s=8)
plt.colorbar()

No linear discriminant power found

In [ ]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression().fit(x,y)
lm.score(x, y)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
preds = RandomForestRegressor(n_estimators=500, n_jobs=-1).fit(x,y).predict(x)

In [ ]:
np.sum((preds - y)**2)

In [ ]:
np.sum((y)**2)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
xcos = cosine_similarity(x, x)

In [ ]:
xcos.shape

In [ ]:
xcos.max()

In [ ]:
xcos.min(), xcos.max()

In [ ]:
plt.imshow(xcos)
plt.colorbar()

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
pcs = PCA(300).fit_transform(RobustScaler().fit_transform(xcos))

In [ ]:
plt.scatter(pcs[:, 0], pcs[:, 1], c=y, s=8)
plt.colorbar()

In [ ]:
cos_preds = RandomForestRegressor(n_estimators=500, n_jobs=-1).fit(pcs,y).predict(pcs)

In [ ]:
np.sum((cos_preds - y)**2)

In [ ]:
from sklearn import tree
treeRegressor = tree.DecisionTreeRegressor(max_leaf_nodes=200, criterion='friedman_mse')
tree_preds = treeRegressor.fit(x, y).predict(x)
np.sum((tree_preds - y)**2)

In [ ]:
from sklearn import tree
treeRegressor = tree.DecisionTreeRegressor(max_leaf_nodes=200, criterion='friedman_mse')
tree_preds = treeRegressor.fit(pcs, y).predict(pcs)
# import graphviz 
# dot_data = tree.export_graphviz(treeRegressor, filled=True, out_file=None) 
# graph = graphviz.Source(dot_data)
# graph.render("gloveUtrsTree") 
print(np.sum((tree_preds - y)**2))


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, np.exp(y), train_size=0.8, random_state=42)
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=50, max_depth=10, oob_score=True, random_state=0, n_jobs=-1)
rf.fit(X_train, y_train)
from sklearn.metrics import r2_score
from scipy.stats import spearmanr, pearsonr
predicted_train = rf.predict(X_train)
predicted_test = np.log(rf.predict(X_test))
test_score = r2_score(np.log(y_test), predicted_test)
spearman = spearmanr(np.log(y_test), predicted_test)
pearson = pearsonr(np.log(y_test), predicted_test)
print(f'Out-of-bag R-2 score estimate: {rf.oob_score_:>5.3}')
print(f'Test data R-2 score: {test_score:>5.3}')
print(f'Test data Spearman correlation: {spearman[0]:.3}')
print(f'Test data Pearson correlation: {pearson[0]:.3}')


In [ ]:
plt.scatter(np.log(y_train), np.log(predicted_train))

In [ ]:
plt.scatter(np.log(y_test), predicted_test)

In [ ]:
import pickle
with open('../processed_data/utrs_glove_embeddings.pkl', 'wb') as out:
    pickle.dump({'data':np.vstack(data['kmers_enc']),'genes':data['gene']}, out)

In [ ]:
!git add ../*

In [ ]:
!git commit -m 'Testing on GLoVe and DF-IDF'

In [ ]:
!git push origin